<a href="https://colab.research.google.com/github/utkarsh027/Chat-Bot/blob/main/PdfQueryLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the 

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [5]:
import os
from constants import openai_key
os.environ["OPENAI_API_KEY"]=openai_key


In [8]:
pdfreader=PdfReader("budget_speech.pdf")

In [9]:
from typing_extensions import Concatenate

raw_text=""
for i , page in enumerate(pdfreader.pages):
  content=page.extract_text()

  if content:
    raw_text+=content

In [10]:
raw_text

"GOVERNMENT OF INDIA\nINTERIM BUDGET 2024-2025\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2024 \nCONTENTS  \n \nPART – A \n Page No.  \nIntroduction  1 \nInclusive Development and Growth  2 \nSocial Justice   3  \nExemplary  Track Record of Governance,  \nDevelopment and Performance (GDP)  7 \nEconomic Management  8 \nGlobal Context  9 \nVision for ‘Viksit Bharat’  10 \nStrategy for  ‘Amrit Kaal’  11 \nInfrastructure Development  17 \nAmrit Kaal as Kartavya Kaal  22 \nRevised Estimates 2023 -24 23 \nBudget Estimates 2024 -25 23 \nPART – B \nDirect taxes  25 \nIndirect Taxes   26 \nEconomy – Then and Now  28 \n  \n  1 \n Interim Budget 2024 -2025  \nSpeech of  \nNirmala Sitharaman  \nMinister of Finance  \nFebruary 1, 2024  \nHon’ble Speaker,  \n I present the Interim Budget for 2024 -25.  \nIntroduction  \n1. The Indian  economy  has witnessed profound positive \ntransformation in the last ten years. The people of India are \nlooking ahead to the future with hop

In [11]:
# We need to split the text using Character Text Split such that it should not increase token size

text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=800,
                                    chunk_overlap=200,
                                    length_function=len,
                                    )

texts=text_splitter.split_text(raw_text)

In [13]:
len(texts)

61

In [14]:
# Download embedding from OpenAI

embeddings=OpenAIEmbeddings()

In [15]:
document_search=FAISS.from_texts(texts,embeddings)

In [16]:
document_search

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [18]:
chain=load_qa_chain(OpenAI(), chain_type="stuff")

In [20]:
query="How much the agriculture target will be increased by"
docs=document_search.similarity_search(query)
chain.run(input_documents=docs, question =query)

' The agriculture target will be increased by five tons per hectare.'